In [2]:
from sklearn.model_selection import train_test_split
import torch
from transformers import GPT2Tokenizer, GPT2Model
from torch.utils import data
import numpy as np

In [3]:
model_name = "gpt2"

In [5]:
data_file = []
with open("../data/bot_detection/train-tweets-POS.txt") as file:
  data_file = file.readlines()

In [6]:
special_tokens = ['@','#','U']
sentences = []
sentence_number = 0
sentences.append([])
for i,line in enumerate(data_file):
    if line!='\n':
        word,POS,prob = line.split('\t')
        POS = POS if (POS in special_tokens) else 'X'
        sentences[sentence_number].append((word,POS))
    else:
        sentence_number = sentence_number + 1
        sentences.append([])
del sentences[-1]

In [7]:
tags = list(set(word_pos[1] for sent in sentences for word_pos in sent))

In [8]:
",".join(tags)

'U,#,@,X'

In [9]:
tags = ["<pad>"] + tags

In [10]:
tag2idx = {tag:idx for idx, tag in enumerate(tags)}
idx2tag = {idx:tag for idx, tag in enumerate(tags)}

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
if model_name == "gpt2":
    special_tokens_dict = {'cls_token': '_classify_','bos_token': '_start_', 'pad_token': '<pad>'}
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2', do_lower_case=True)
    tokenizer.add_special_tokens(special_tokens_dict)

In [15]:
class PosDataset(data.Dataset):
    def __init__(self, tagged_sents):
        sents, tags_li = [], [] # list of lists
        for sent in tagged_sents:
            words = [word_pos[0] for word_pos in sent]
            tags = [word_pos[1] for word_pos in sent]
            if model_name == "gpt2":
                sents.append(["_start_"] + words + ["_classify_"])
                tags_li.append(["<pad>"] + tags + ["<pad>"])
            else:
                sents.append(["[CLS]"] + words + ["[SEP]"])
                tags_li.append(["<pad>"] + tags + ["<pad>"])
        self.sents, self.tags_li = sents, tags_li

    def __len__(self):
        return len(self.sents)

    def __getitem__(self, idx):
        words, tags = self.sents[idx], self.tags_li[idx] # words, tags: string list

        # We give credits only to the first piece.
        x, y = [], [] # list of ids
        is_heads = [] # list. 1: the token is the first piece of a word
        for w, t in zip(words, tags):
            tokens = tokenizer.tokenize(w) if w not in ("[CLS]", "[SEP]") else [w]
            if(len(tokens)==0): 
                tokens=[w]
            xx = tokenizer.convert_tokens_to_ids(tokens)

            is_head = [1] + [0]*(len(tokens) - 1)

            t = [t] + ["<pad>"] * (len(tokens) - 1)  # <PAD>: no decision
            yy = [tag2idx[each] for each in t]  # (T,)

            x.extend(xx)
            is_heads.extend(is_head)
            y.extend(yy)
        if(not len(x)==len(y)==len(is_heads)):
            print(words, tags)
            print(x,y,is_heads)
        assert len(x)==len(y)==len(is_heads), "len(x)={}, len(y)={}, len(is_heads)={}".format(len(x), len(y), len(is_heads))

        # seqlen
        seqlen = len(y)

        # to string
        words = " ".join(words)
        tags = " ".join(tags)
        return words, x, is_heads, tags, y, seqlen

In [13]:
def pad(batch):
    '''Pads to the longest sample'''
    f = lambda x: [sample[x] for sample in batch]
    words = f(0)
    is_heads = f(2)
    tags = f(3)
    seqlens = f(-1)
    maxlen = np.array(seqlens).max()

    f = lambda x, seqlen: [sample[x] + [0] * (seqlen - len(sample[x])) for sample in batch] # 0: <pad>
    x = f(1, maxlen)
    y = f(-2, maxlen)


    f = torch.LongTensor

    return words, f(x), is_heads, tags, f(y), seqlens

In [16]:
dataset = PosDataset(sentences)
data_iter = data.DataLoader(dataset=dataset,
                             batch_size=8,
                             shuffle=True,
                             num_workers=1,
                             collate_fn=pad)

In [23]:
if model_name == "gpt2":
    #model = GPT2Model.from_pretrained("gpt2")
    model = torch.load("../models/Gpt2_Classifier_Large.pt")
    model = model.transformer
    model.output_hidden_states = True
    model.resize_token_embeddings(len(tokenizer))
model = model.to(device)


In [24]:
from sklearn.cluster import KMeans

In [25]:
from sklearn.metrics.cluster import normalized_mutual_info_score

Pretrained:

In [20]:
all_nmi =[]
for n in range(0,10):
    Tags, Embedds = [], [[] for i in range(0,13)]
    #Tags, Embedds = [], []

    for i, batch in enumerate(data_iter):
        _, x, is_heads_b, _, y_b, _ = batch
        x = x.to(device)
        with torch.no_grad():
            output=model(x)
            embedds_b = output[2]
            #embedds_b = output[0] # We want to test output embeddings
            for i in range(0,13):
                for embedds, is_heads, ys in zip(embedds_b[i].cpu().numpy(), is_heads_b, y_b):
                    embbed = [hat for head, hat in zip(is_heads, embedds) if head == 1]   
                    y = [hat.item() for head, hat in zip(is_heads, ys) if head == 1]  
                    Embedds[i].extend(embbed[1:-1])
                    if(i==0):
                        Tags.extend(y[1:-1])
            #for embedds, is_heads, ys in zip(embedds_b.cpu().numpy(), is_heads_b, y_b):
                #embbed = [hat for head, hat in zip(is_heads, embedds) if head == 1]   
                #y = [hat.item() for head, hat in zip(is_heads, ys) if head == 1]  
                #Embedds.extend(embbed[1:-1])
                #if(i==0):
                    #Tags.extend(y[1:-1])
                #Tags.extend(y[1:-1])
            if(len(Embedds[i])>=3000):
            #if(len(Embedds)>=3000):
                break
    nmi=[]
    for i in range(0,13):
        kmeans = KMeans(n_clusters=4, random_state=0).fit(Embedds[i])
        nmi.append(normalized_mutual_info_score(np.array(Tags),kmeans.labels_))
    all_nmi.append(nmi)
    print(n)

0
1
2
3
4
5
6
7
8
9


In [21]:
np.mean(np.array(all_nmi),axis=0)

array([0.26615675, 0.33489916, 0.44400374, 0.50212925, 0.49766814,
       0.47753819, 0.45593526, 0.40627995, 0.33742857, 0.34303634,
       0.32778245, 0.28541799, 0.23972654])

Finetuned:

In [26]:
all_nmi =[]
for n in range(0,10):
    Tags, Embedds = [], [[] for i in range(0,13)]
    #Tags, Embedds = [], []

    for i, batch in enumerate(data_iter):
        _, x, is_heads_b, _, y_b, _ = batch
        x = x.to(device)
        with torch.no_grad():
            output=model(x)
            embedds_b = output[2]
            #embedds_b = output[0] # We want to test output embeddings
            for i in range(0,13):
                for embedds, is_heads, ys in zip(embedds_b[i].cpu().numpy(), is_heads_b, y_b):
                    embbed = [hat for head, hat in zip(is_heads, embedds) if head == 1]   
                    y = [hat.item() for head, hat in zip(is_heads, ys) if head == 1]  
                    Embedds[i].extend(embbed[1:-1])
                    if(i==0):
                        Tags.extend(y[1:-1])
            #for embedds, is_heads, ys in zip(embedds_b.cpu().numpy(), is_heads_b, y_b):
                #embbed = [hat for head, hat in zip(is_heads, embedds) if head == 1]   
                #y = [hat.item() for head, hat in zip(is_heads, ys) if head == 1]  
                #Embedds.extend(embbed[1:-1])
                #if(i==0):
                    #Tags.extend(y[1:-1])
                #Tags.extend(y[1:-1])
            if(len(Embedds[i])>=3000):
            #if(len(Embedds)>=3000):
                break
    nmi=[]
    for i in range(0,13):
        kmeans = KMeans(n_clusters=4, random_state=0).fit(Embedds[i])
        nmi.append(normalized_mutual_info_score(np.array(Tags),kmeans.labels_))
    all_nmi.append(nmi)
    print(n)

0
1
2
3
4
5
6
7
8
9


In [27]:
np.mean(np.array(all_nmi),axis=0)

array([0.18091549, 0.35020918, 0.38046554, 0.4527303 , 0.43411033,
       0.51854905, 0.45723204, 0.37932747, 0.33658722, 0.31705283,
       0.29594284, 0.16537291, 0.05815219])